<a href="https://colab.research.google.com/github/JeisonTantachuco/CursoGEE/blob/main/module06/02_ICviz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://user-images.githubusercontent.com/16768318/73986808-75b3ca00-4936-11ea-90f1-3a6c352766ce.png" width=10% >
<img align="right" style="padding-left:10px;" src="https://user-images.githubusercontent.com/16768318/73986811-764c6080-4936-11ea-9653-a3eacc47caed.png" width=10% >

**Bienvenidos!** Este *colab notebook* es parte del curso [**Introduccion a Google Earth Engine con Python**](https://github.com/csaybar/EarthEngineMasterGIS) desarrollado por el equipo [**MasterGIS**](https://www.mastergis.com/). Obten mas informacion del curso en este [**enlace**](https://www.mastergis.com/product/google-earth-engine-python/). El contenido del curso esta disponible en [**GitHub**](https://github.com/csaybar/EarthEngineMasterGIS) bajo licencia [**MIT**](https://opensource.org/licenses/MIT).

## **MASTERGIS: Visualizacion de un ee.ImageCollection**

En esta lectura, aprenderemos sobre como realizar vizualizaciones en colecciones de imagenes.

In [1]:
#@title Credenciales Google Earth Engine
import ee
ee.Authenticate()
ee.Initialize(project = 'ee-jtantaroman')

In [2]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [ ]:
# !pip install earthengine-api==0.1.210 esto ya no es necesario en ultima versiones

### **Ejercicio**

Las animaciones de series temporales de imagenes satelitales son una forma atractivas de mostrar tus resultados. En esta seccion, aprenderemos como generar un GIF animado que represente un **NDVI promedio de 20 anos para compuestos MODIS de 16 dias que abarcan del 1 de enero al 31 de diciembre**.
<center>
<img src='https://user-images.githubusercontent.com/16768318/73617762-142fea80-461a-11ea-8c8a-0a124a0735c1.gif'>
</center>

#### **Contexto**

**MODIS** es un sistema de imagenes satelitales de resolución moderada y el **NDVI** es un indice de vegetacion basado en bandas reflectancia. En la animacion superior, el NDVI se asigna a un gradiente de color de marron a verde oscuro, representando una capacidad fotosintetica baja a alta respectivamente.

**Instrucciones**

El flujo de trabajo basico para realizar una animacion es el siguiente:

1. Definir region de interes.

2. Obtenga el conjunto de datos de indices de vegetación MODIS y seleccione la banda de NDVI.

3. **Agrupe las imagenes de la misma ventana temporal (16 dias) utilizando un _join_**.

4. Reduzca el grupo de compuestos de NDVI (por la media) para producir la animacion.

5. Defina los parametros de visualizacion y convierta los datos en imagenes de visualizacion RGB.

6. Genere una URL que produzca un GIF en el navegador

### **1. Area de estudio**
La variable **mask** sera usada para recortar las imagenes, este es un paso opcional que establece los pixeles fuera de un area de interes como nulos, lo que es util para exhaltar los valores sobre un area particular dentro de una region. Aqui, el conjunto de datos vectoriales **LSIB** se filtra para incluir solo la geometria de Peru (que servira como geometria de recorte).

**geom_Peru** es utilizado dentro de _ee.ImageCollection.filterBounds()_.

In [3]:
#Los limites de los paises ya se encuentra en una capa en la bd de gee
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
         .filter(ee.Filter.eq('country_co', 'PE'))
geom_peru = mask.geometry().bounds() #Poligono que contiene los limites norte,sur,este y oeste del peru

In [5]:
mapdisplay([-76,-12],{'peru':mask.getInfo()})

### **2. Filtrando dataset**
**ee.Date.getRelative(...)**: Devuelve la diferencia (considerando desde 0) de esta fecha en relación con una unidad mas grande, por ejemplo, **getRelative('day', 'year')** devuelve un valor entre 0 y 365.

In [6]:
# Cargamos los datos de NDVI de MODIS
col = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI')

# Agregamos la propiedad day-of-year (DOY) para cada imagen
# GEE trabaja las fechs utilizando time stamp (utilizando como fecha de partida a 01-01-1970), pero Modis utiliza dias julianos es decir del 0 al 365 y el año al costado ejem
# 32 2010 vendria a ser el 01-02-2010
def add_day(img):
  doy = ee.Date(img.get('system:time_start')).getRelative('day', 'year') #Transforma las fechas de 0 a 365 (porque se esta utilizando como parametro superior year, si fuera week seria de 1 al 7)
  return img.set('doy', doy)

col = col.map(add_day)
col.first().getInfo()
#Doy : 48, fecha de la primera imagen es 17 de febrero

/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD13A2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A2

  warnings.warn(warning, category=DeprecationWarning)


{'type': 'Image',
 'bands': [{'id': 'NDVI',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [43200, 21600],
   'crs': 'SR-ORG:6974',
   'crs_transform': [926.625433056,
    0,
    -20015109.354,
    0,
    -926.625433055,
    10007554.677]}],
 'version': 1638764935442440.0,
 'id': 'MODIS/006/MOD13A2/2000_02_18',
 'properties': {'doy': 48,
  'system:time_start': 950832000000,
  'google:max_source_file_timestamp': 1528310897320,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 952214400000,
  'system:asset_size': 2806855579,
  'system:index': '2000_02_18'}}

### **3. Join**

**Join en GEE nos permite agrupar imagenes o features mediante propiedades**. Para este ejercicio, se implementara una operacion **join** para agrupar dos grupos de imagenes (ImageCollection) bajo la propiedad **'doy'** que acabamos de generar en la celda anterior.

**ee.join** siempre espera dos colecciones: una **coleccion de referencia** y una **coleccion completa**, que al final sera modificada para incluir las propiedades de la **coleccion de referencia**. La coleccion completa (col) ya existe, la **colección de referencia** necesita ser definida. Para ello, filtre la coleccion completa en un solo año de datos (no importa el año que escoja), por ejemplo 2013.

Para crear un **ee.join** considere:

1. Crear un filtro que identificara imagenes coincidentes entre la **coleccion de referencia** y **coleccion completa** por la propiedad **doy**.
2. Defina un **saveALL join**  que producira una lista de todas las coincidencias por **'doy'** distintas en una propiedad llamada **'doy_matches'**.
3. Aplicando la **join**.

**TENER EN CUENTA QUE luego de aplicar un ee.Join.SaveAll el resultado siempre sera un ee.FeatureCollection**

In [9]:
# Obtenga una colección de imagenes distintas de 'doy' (COLLECCION DE REFERENCIA).
distinctDOY = col.filterDate('2013-01-01', '2014-01-01')

# Defina un filtro que identifique que imagenes de la 'coleccion completa'
# coincide con el DOY de la colección de referencia.
my_filter = ee.Filter.equals(leftField = 'doy', rightField = 'doy')

# Define el Join
join = ee.Join.saveAll('doy_matches')

#Visualizar el resultado, el resultado siempre va a ser un feature collection pero sin geometrias
join.apply(distinctDOY, col, my_filter).first().getInfo()

#ctrl + f y busca system:index, vamos a encontrar que nuestra base de datos de referencia que empezó con 2013-01-01 (doy 0) ha hecho match con todas las imagenes de doy0 de la base de datos general y con el system index veremos que son los 01-01 de todos los años

{'type': 'Image',
 'bands': [{'id': 'NDVI',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [43200, 21600],
   'crs': 'SR-ORG:6974',
   'crs_transform': [926.625433056,
    0,
    -20015109.354,
    0,
    -926.625433055,
    10007554.677]}],
 'version': 1638765084595611,
 'id': 'MODIS/006/MOD13A2/2013_01_01',
 'properties': {'doy_matches': [{'type': 'Image',
    'bands': [{'id': 'NDVI',
      'data_type': {'type': 'PixelType',
       'precision': 'int',
       'min': -32768,
       'max': 32767},
      'dimensions': [43200, 21600],
      'crs': 'SR-ORG:6974',
      'crs_transform': [926.625433056,
       0,
       -20015109.354,
       0,
       -926.625433055,
       10007554.677]}],
    'version': 1638764945654270.0,
    'id': 'MODIS/006/MOD13A2/2001_01_01',
    'properties': {'doy': 0,
     'system:time_start': 978307200000,
     'google:max_source_file_timestamp': 1528309231052,
     'system:footprint': {'type': 

In [10]:

# Los resultados no se guardan en bandas, se guardan en propiedades, lo que se debe hacer es extraer esas propiedades
# Para eso primero pasaremos el feature collection a image collection para luego aplicarle un map, y extraer las propiedades doy_matches que es una lista de imagenes que hizo match

# Aplicar el join y convertir el FeatureCollection resultante en un ImageCollection.
joinCol = ee.ImageCollection(join.apply(distinctDOY, col, my_filter))

# Aplicar la mediana entre las colecciones DOY coincidentes.
def reduce_each_image(img):
  doyCol = ee.ImageCollection.fromImages(img.get('doy_matches')) #Aca se esta generando el image collection de la lista de imagenes, exiten listas para cada grupo que hizo match, doy = 0 (imagenes del 01-01 de todos los años), doy = 16 (imagenes del 16-01 de todos los años), etc ..
  return doyCol.reduce(ee.Reducer.median())  #Aca se genera un image (agrupamos cada lista de los doy 0, 16 ,etc; la agrupacion saca la mediana (imagen de una sola banda ya no image collection)), pero serian la n cantidad de imagenes igual a la cantidad de doys en el año 0,16,32 etc.

comp = joinCol.map(reduce_each_image)

#### **4. Hacer imagenes de visualizacion**

Establezca las propiedades de visualizacion (**ee.Image.vizualize**) para controlar la apariencia de los datos en los cuadros de animacion. Defina las propiedades de visualizacion que incluyen una paleta de colores y valores minimos y maximos para estirar o comprimir la paleta. Asigne el metodo de visualizacion sobre cada imagen utilizando las propiedades de visualizacion definidas y recorte los datos a la mascara definida en el seccin 1 para establecer los pixeles de fondo nulo (negro).

In [11]:
def vizparam(img):
  # Parametros de vizualicion.
  return img.visualize(
       min = 0.0,
       max = 9000.0,
       palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
                  '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
                  '012E01', '011D01', '011301']
                ).clip(mask)

### **5. Crea un GIF animado**

El paso final es definir argumentos para el método **getVideoThumbURL** que creara una animación a partir de la colección de imágenes de visualización RGB generadas en el paso anterior. Los parámetros relevantes incluyen:

region: establecer en la región definida en el Paso 2 anterior
dimensiones: establecido en 600 que define la dimensión más grande del GIF resultante como 600px (la dimensión más pequeña se escala proporcionalmente)
crs: establecido en 'EPSG: 3857' para que coincida con el sistema de referencia de coordenadas del mapa del editor de código
framesPerSecond: establecido en 10 (cuadros por segundo)

In [12]:
# Create RGB visualization images for use as animation frames.
rgbVis = comp.map(vizparam)

# Define GIF visualization arguments.
gifParams = {
  'region': geom_peru,
  'dimensions': 600,
  'crs': 'EPSG:4326',
  'framesPerSecond': 4,
  'format': 'gif'
}

# Print the GIF URL to the console.
gif = rgbVis.getVideoThumbURL(gifParams)

In [13]:
from IPython.display import Image
from IPython.core.display import HTML
Image(url= gif)

#### **Funcion para crear animaciones de NDVI en cualquier parte del mundo**

In [14]:
def monthly_ndvi_animation(area, vizparams = None, framesPerSecond = 10):
  if vizparams is None:
    vizparams = {'min' : 0.0,
                 'max' : 9000.0,
                 'palette' : ['#051852', '#FFFFFF', '#C7B59B', '#A8B255',
                              '#A3C020', '#76AD00', '#429001', '#006400',
                              '#003B00', '#000000']
                 }
  # Cargamos los datos de NDVI de MODIS
  col = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI')

  # Agregamos la propiedad day-of-year (DOY) para cada imagen
  def add_day(img):
    doy = ee.Date(img.get('system:time_start')).getRelative('day', 'year')
    return img.set('doy', doy)

  col = col.map(add_day)

  # Obtenga una colección de imágenes distintas de 'doy'.
  distinctDOY = col.filterDate('2013-01-01', '2014-01-01')

  # Defina un filtro que identifique que imagenes de la 'coleccion completa'
  # coincide con el DOY de la colección de referencia.
  my_filter = ee.Filter.equals(leftField = 'doy', rightField = 'doy')

  # Define el Join
  join = ee.Join.saveAll('doy_matches')

  # Aplicar el join y convertir el FeatureCollection resultante en un ImageCollection.
  joinCol = ee.ImageCollection(join.apply(distinctDOY, col, my_filter))

  # Aplicar la mediana entre las colecciones DOY coincidentes.
  def reduce_each_image(img):
    doyCol = ee.ImageCollection.fromImages(img.get('doy_matches'))
    return doyCol.reduce(ee.Reducer.median())

  comp = joinCol.map(reduce_each_image)

  # Cree imagenes de visualizacion RGB para usar como cuadros de animacion.
  rgbVis = comp.map(lambda x:x.visualize(**vizparams).clip(area))

  # Definir argumentos de visualización GIF.
  gifParams = {
    'region': area.bounds(),
    'dimensions': 600,
    'crs': 'EPSG:3857',
    'framesPerSecond': framesPerSecond,
    'format': 'gif'
  }
  # Imprima la URL GIF en la consola.
  return rgbVis.getVideoThumbURL(gifParams)

In [16]:
ecuador = ee.Geometry.Polygon([
          [
            [
              -82.37548828125,
              -5.7908968128719565
            ],
            [
              -74.86083984375,
              -5.7908968128719565
            ],
            [
              -74.86083984375,
              1.9551868337565232
            ],
            [
              -82.37548828125,
              1.9551868337565232
            ],
            [
              -82.37548828125,
              -5.7908968128719565
            ]
          ]
        ])
monthly_ndvi_animation(ecuador, framesPerSecond = 5)

'https://earthengine.googleapis.com/v1/projects/ee-jtantaroman/videoThumbnails/924f7dd4bf928fde582a435364aa4b2f-be634e54a4f9071c75495247d3389666:getPixels'

### **¿Dudas con este Jupyer-Notebook?**

Estaremos felices de ayudarte!. Create una cuenta Github si es que no la tienes, luego detalla tu problema ampliamente en: https://github.com/csaybar/EarthEngineMasterGIS/issues

**Tienes que dar clic en el boton verde!**

<center>
<img src="https://user-images.githubusercontent.com/16768318/79680748-d5511000-81d8-11ea-9f89-44bd010adf69.png" width = 70%>
</center>